In [ ]:
# %matplotlib ipympl
from datetime import timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from plotting import *
from processing import *

In [ ]:
MUNICIPAL_YEARS = np.arange(2011, 2023, 2)
STATE_YEARS = np.arange(2012, 2023, 2)
NATIONAL_YEARS = np.arange(2012, 2023, 4)
ALL_YEARS = np.arange(2011, 2023, 1)
voters = load_full_dataset("data")

In [ ]:
voters

In [ ]:
voters.loc[pd.IndexSlice[:, "03CTS1671000"], :]

In [ ]:
new_df = (
    pd.DataFrame(
        {
            "local_elections": voters.loc[MUNICIPAL_YEARS]
            .reset_index()["Voter ID Number"]
            .value_counts(),
            "local_votes": voters.loc[MUNICIPAL_YEARS]
            .groupby("Voter ID Number")["voted"]
            .sum(),
            "state_elections": voters.loc[STATE_YEARS]
            .reset_index()["Voter ID Number"]
            .value_counts(),
            "state_votes": voters.loc[STATE_YEARS]
            .groupby("Voter ID Number")["voted"]
            .sum(),
        },
        dtype=int,
    )
    .fillna(0)
    .astype(int)
    .sort_values(["local_elections"], ascending=False)
)
# voters.loc[MUNICIPAL_YEARS].reset_index().groupby("Voter ID Number")["voted"].sum()

In [ ]:
unique_ids = voters.reset_index()["Voter ID Number"].unique()
# there must be a better way
# but idk what that would be, so here we are
most_recent_local = pd.Series(index=unique_ids)
most_recent_state = pd.Series(index=unique_ids)

for year in MUNICIPAL_YEARS:
    most_recent_local.loc[
        voters.loc[year][voters.loc[year]["voted"] == True].index
    ] = year
for year in STATE_YEARS:
    print(year)
    most_recent_state.loc[
        voters.loc[year][voters.loc[year]["voted"] == True].index
    ] = year

In [ ]:
most_recent_state = most_recent_state.dropna().astype(int)
most_recent_local = most_recent_local.dropna().astype(int)

In [ ]:
cur_registered_df = pd.read_csv(
    "data/49VOT_281914.txt", delimiter="|", encoding="ISO-8859-1"
)
cur_registered_df.rename(
    {c: c.strip() for c in cur_registered_df.columns}, inplace=True, axis=1
)
cur_registered_df.rename(
    {
        "Mailing Address ¿ Street Number and Name": "Mailing Address - Street Number and Name"
    },
    inplace=True,
    axis=1,
)
birth_dates = pd.to_datetime(cur_registered_df["Date of Birth"])

# upcoming election date
elec_date = pd.to_datetime("11/07/2023")

# # check that this is actually correct ideally
# subtract (1-1/365) to make it so people turning 18 on election day count as 18
age = np.ceil(
    (elec_date - birth_dates) / timedelta(days=365.2425) - (1 - 1 / 365)
).astype(int)
cur_registered_df["age"] = age
cur_registered_df = cur_registered_df.set_index("Voter ID Number")

In [ ]:
new_df["most_recent_local"] = most_recent_local
new_df["most_recent_state"] = most_recent_state

In [ ]:
cur_reg_hist = new_df.loc[
    list(set(cur_registered_df.index).intersection(set(new_df.index)))
]
base_cols = [
    "Last Name",
    "First Name",
    "Middle Name",
    "age",
    "Date of Birth",
    "Date of Registration",
    "Residential Address Street Number",
    "Residential Address Street Name",
    "Residential Address Street Suffix",
    "Residential Address Apartment Number",
    "Residential Address Zip Code",
    "Mailing Address - Street Number and Name",
    "Mailing Address - Apartment Number",
    "Mailing Address - City or Town",
    "Mailing Address - State",
    "Mailing Address - Zip Code",
    "Voter Status",
    "Party Affiliation",
    "Ward Number",
    "Precinct Number",
    "Congressional District Number",
    "Senatorial District Number",
    "State Representative District",
]
cur_reg_hist = cur_reg_hist.merge(
    cur_registered_df[base_cols], left_on="Voter ID Number", right_on="Voter ID Number"
)
cols = [
    "local_elections",
    "local_votes",
    "state_elections",
    "state_votes",
    "most_recent_local",
    "most_recent_state",
] + base_cols
cur_reg_hist = cur_reg_hist[cols].sort_values(
    ["local_votes", "local_elections"], ascending=False
)
cur_reg_hist

In [ ]:
cur_reg_hist[cur_reg_hist["Last Name"].isin(["AZEEM", "HUNT-ISAAK"])]

In [ ]:
cur_reg_hist.to_csv("individual_voter_voting_frequencies.csv")

In [ ]:
cur_registered_df[
    pd.to_datetime(cur_registered_df["Date of Registration"])
    > pd.to_datetime("2022-11-08")
][base_cols].to_csv("registered_since_last_election.csv")

In [ ]:
def at_address(df, street_number, street_name):
    return df[
        (df["Residential Address Street Name"] == street_name)
        & (df["Residential Address Street Number"] == street_number)
    ]

In [ ]:
grp = cur_registered_df.groupby(
    [
        "Residential Address Street Name",
        "Residential Address Street Number",
        # "Residential Address Apartment Number"
    ]
)
# .size()).rename({0:"Registered Voters"},axis=1).sort_values("Registered Voters", ascending =False)

In [ ]:
# grp.agg(registered="size")
# df.groupby("A").agg(
out = grp.agg(
    registered=pd.NamedAgg(column="Record Sequence Number", aggfunc="size"),
    avg_age=pd.NamedAgg(column="age", aggfunc="mean"),
    age_std=pd.NamedAgg(column="age", aggfunc="std"),
    ward=pd.NamedAgg(column="Ward Number", aggfunc="median"),
).sort_values(["ward", "registered"], ascending=[True, False])

out["ward"] = out["ward"].astype(int)
# for ward in sorted(out['ward'].unique()):
#     out['ward']
out.to_csv("most-registered-addresses-with-ward.csv", float_format="%.2f")

In [ ]:
cur_reg_hist

In [ ]:
out = (
    cur_reg_hist.groupby(
        [
            "Residential Address Street Name",
            "Residential Address Street Number",
            # "Residential Address Apartment Number"
        ]
    )
    .agg(
        local_elections_sum=pd.NamedAgg(column="local_elections", aggfunc="sum"),
        local_votes_sum=pd.NamedAgg(column="local_votes", aggfunc="sum"),
        state_elections_sum=pd.NamedAgg(column="state_elections", aggfunc="sum"),
        state_votes_sum=pd.NamedAgg(column="state_votes", aggfunc="sum"),
        most_recent_local_avg=pd.NamedAgg(column="most_recent_local", aggfunc="mean"),
        most_recent_state_avg=pd.NamedAgg(column="most_recent_state", aggfunc="mean"),
        registered=pd.NamedAgg(column="age", aggfunc="size"),
        avg_age=pd.NamedAgg(column="age", aggfunc="mean"),
        age_std=pd.NamedAgg(column="age", aggfunc="std"),
        ward=pd.NamedAgg(column="Ward Number", aggfunc="median"),
    )
    .sort_values(["ward", "registered"], ascending=[True, False])
)

out["ward"] = out["ward"].astype(int)
out["local_turnout_perc"] = 100 * out["local_votes_sum"] / out["local_elections_sum"]
out["state_turnout_perc"] = 100 * out["state_votes_sum"] / out["state_elections_sum"]
out.index = out.index.rename(
    {
        "Residential Address Street Name": "street name",
        "Residential Address Street Number": "street number",
    },
)
cols = (
    list(out.columns[:4])
    + ["local_turnout_perc", "state_turnout_perc"]
    + list(out.columns[4:-2])
)
out = out[cols]
out.reorder_levels([1, 0]).to_csv(
    "most-reg-addresses-with-turnout.csv", float_format="%.2f"
)

In [ ]:
out

In [ ]:
out.reset_index().columns

In [ ]:
out.reset_index()

In [ ]:
at_address(cur_registered_df, 2, "2ND ST")

In [ ]:
at_address(cur_registered_df, 4, "2ND ST")

In [ ]:
s = set(cur_registered_df.index)
# .intersection(new_df.index))].sort_values(["local_elections", "local_votes"], ascending=False)

In [ ]:
cur_registered_df.loc[list(set(cur_registered_df.index).difference(new_df.index))]

In [ ]:
# all people currently registered but who have never had the opportunity to vote
print("all people currently registered but who have never had the opportunity to vote")
len(list(s.difference(new_df.index)))

In [ ]:
cur_registered_df.set_index("Voter ID Number")

In [ ]:
new_df = new_df.loc[
    list(set(cur_registered_df["Voter ID Number"]).intersection(new_df.index))
].sort_values(["local_elections", "local_votes"], ascending=False)